<a href="https://colab.research.google.com/github/YasirHabib/Deep-Learning-Advanced-Computer-Vision/blob/master/conv_block_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [0]:
import tensorflow as tf

In [0]:
def init_filter(shape):
  w = np.random.randn(*shape) * np.sqrt(2.0 / np.prod(shape[:-1]))
  return w.astype(np.float32)

In [0]:
class ConvLayer():
  def __init__(self, num_feature_maps, num_color_channels, filter_width, filter_height, stride=2, padding='VALID'):
    sz = (filter_width, filter_height, num_color_channels, num_feature_maps)
    W = init_filter(sz)
    b = np.zeros(num_feature_maps, dtype=np.float32)

    self.W = tf.Variable(W)
    self.b = tf.Variable(b)
    self.stride = stride
    self.padding = padding
    
  def ConvLayer_forward(self, X):
    conv_out = tf.nn.conv2d(X, self.W, strides=[1, self.stride, self.stride, 1], padding=self.padding)
    conv_out = tf.nn.bias_add(conv_out, self.b)
    return conv_out
  
  def copyFromKerasLayers(self, layer):
    # only 1 layer to copy from
    W, b = layer.get_weights()
    op1 = self.W.assign(W)
    op2 = self.b.assign(b)
    self.session.run((op1, op2))

  #def copyFromWeights(self, W, b):
  # op1 = self.W.assign(W)
  # op2 = self.b.assign(b)
  # self.session.run((op1, op2))

  def get_params(self):
    return [self.W, self.b]

In [0]:
class BatchNormLayer():
	def __init__(self, D):
    self.running_mean = tf.Variable(np.zeros(D, dtype=np.float32), trainable=False)
    self.running_var  = tf.Variable(np.ones(D, dtype=np.float32), trainable=False)
    self.gamma        = tf.Variable(np.ones(D, dtype=np.float32))
    self.beta         = tf.Variable(np.zeros(D, dtype=np.float32))

	def BatchNormLayer_forward(self, X):
    return tf.nn.batch_normalization(X, self.running_mean, self.running_var, self.beta, self.gamma, 1e-3)
    
  def copyFromKerasLayers(self, layer):
    # only 1 layer to copy from
    # order:
    # gamma, beta, moving mean, moving variance
    gamma, beta, running_mean, running_var = layer.get_weights()
    op1 = self.running_mean.assign(running_mean)
    op2 = self.running_var.assign(running_var)
    op3 = self.gamma.assign(gamma)
    op4 = self.beta.assign(beta)
    self.session.run((op1, op2, op3, op4))

  def get_params(self):
    return [self.running_mean, self.running_var, self.gamma, self.beta]